In [1]:
# 경고 메세지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold
from sklearn.model_selection import KFold           # 랜덤하게 섞어서 자를 수도 있고, 순서대로 자를 수도 있음
from sklearn.model_selection import StratifiedKFold # 결과 데이터의 비율이 최대한 균등하게 들어갈 수 있도록!

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 다중분류를 위한 핫-윈 인코더
from keras.utils import to_categorical

# 저장된 딥러닝 모델을 복구하는 함수
from keras.models import load_model

# epoch마다 모델을 저장하는 함수
from keras.callbacks import ModelCheckpoint

# 더이상 성능 향상이 이루어지지 않는다면 조기 중단시킬 수 있는 함수
from keras.callbacks import EarlyStopping

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family']='Malgun Gothic'
# 맥용 plt.rcParams['font.family']='AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터 입력
df1 = pd.read_csv('../../data/wine.csv', header=None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [4]:
# 데이터 분류
x = df1.drop(12, axis=1)
y = df1[12]

display(x)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [5]:
# 모델 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [6]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# 모델 저장 콜백
modelpath = 'models/{epoch}-{val_loss}.hdf5'
callback1 = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [8]:
# 중단 콜백
# patience : 값을 조절하면서 확인 해보기

callback2 = EarlyStopping(monitor='val_loss', patience=100)

In [9]:
# 모델 실행
# 어차피 조기 중단 되니까 에포크 많이 주기

model.fit(x, y, validation_split=0.2, epochs=100000, batch_size=500, callbacks=[callback1, callback2])

Epoch 1/100000
11/11 [==============================] - 1s 33ms/step - loss: 1.7230 - accuracy: 0.6724 - val_loss: 0.6574 - val_accuracy: 0.6769

Epoch 00001: val_loss improved from inf to 0.65745, saving model to models\1-0.6574481725692749.hdf5
Epoch 2/100000
11/11 [==============================] - 0s 11ms/step - loss: 0.6426 - accuracy: 0.6536 - val_loss: 0.0777 - val_accuracy: 0.9954

Epoch 00002: val_loss improved from 0.65745 to 0.07766, saving model to models\2-0.07766126841306686.hdf5
Epoch 3/100000
11/11 [==============================] - 0s 5ms/step - loss: 0.4765 - accuracy: 0.7516 - val_loss: 0.2655 - val_accuracy: 0.9246

Epoch 00003: val_loss did not improve from 0.07766
Epoch 4/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.3877 - accuracy: 0.8473 - val_loss: 0.1922 - val_accuracy: 0.9585

Epoch 00004: val_loss did not improve from 0.07766
Epoch 5/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.3404 - accuracy: 0.8888 - val_l

11/11 [==============================] - 0s 6ms/step - loss: 0.1794 - accuracy: 0.9361 - val_loss: 0.1149 - val_accuracy: 0.9738

Epoch 00042: val_loss did not improve from 0.07766
Epoch 43/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1724 - accuracy: 0.9373 - val_loss: 0.1196 - val_accuracy: 0.9731

Epoch 00043: val_loss did not improve from 0.07766
Epoch 44/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1785 - accuracy: 0.9337 - val_loss: 0.0946 - val_accuracy: 0.9792

Epoch 00044: val_loss did not improve from 0.07766
Epoch 45/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1763 - accuracy: 0.9358 - val_loss: 0.1020 - val_accuracy: 0.9754

Epoch 00045: val_loss did not improve from 0.07766
Epoch 46/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1683 - accuracy: 0.9379 - val_loss: 0.1194 - val_accuracy: 0.9723

Epoch 00046: val_loss did not improve from 0.07766
Epoch 47/100000
11/11 [========

11/11 [==============================] - 0s 6ms/step - loss: 0.1184 - accuracy: 0.9617 - val_loss: 0.0750 - val_accuracy: 0.9823

Epoch 00082: val_loss did not improve from 0.04227
Epoch 83/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1148 - accuracy: 0.9636 - val_loss: 0.0450 - val_accuracy: 0.9892

Epoch 00083: val_loss did not improve from 0.04227
Epoch 84/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1266 - accuracy: 0.9534 - val_loss: 0.0393 - val_accuracy: 0.9900

Epoch 00084: val_loss improved from 0.04227 to 0.03926, saving model to models\84-0.039262305945158005.hdf5
Epoch 85/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1215 - accuracy: 0.9562 - val_loss: 0.0556 - val_accuracy: 0.9877

Epoch 00085: val_loss did not improve from 0.03926
Epoch 86/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.1112 - accuracy: 0.9651 - val_loss: 0.0564 - val_accuracy: 0.9877

Epoch 00086: val_loss did


Epoch 00122: val_loss did not improve from 0.03017
Epoch 123/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0814 - accuracy: 0.9737 - val_loss: 0.0329 - val_accuracy: 0.9885

Epoch 00123: val_loss did not improve from 0.03017
Epoch 124/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0839 - accuracy: 0.9709 - val_loss: 0.0395 - val_accuracy: 0.9869

Epoch 00124: val_loss did not improve from 0.03017
Epoch 125/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0862 - accuracy: 0.9712 - val_loss: 0.0408 - val_accuracy: 0.9854

Epoch 00125: val_loss did not improve from 0.03017
Epoch 126/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0878 - accuracy: 0.9715 - val_loss: 0.0547 - val_accuracy: 0.9831

Epoch 00126: val_loss did not improve from 0.03017
Epoch 127/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0829 - accuracy: 0.9728 - val_loss: 0.0529 - val_accuracy: 0.9838

Epoch 001

11/11 [==============================] - 0s 6ms/step - loss: 0.0788 - accuracy: 0.9762 - val_loss: 0.0291 - val_accuracy: 0.9923

Epoch 00164: val_loss did not improve from 0.02607
Epoch 165/100000
11/11 [==============================] - 0s 11ms/step - loss: 0.0719 - accuracy: 0.9768 - val_loss: 0.0299 - val_accuracy: 0.9915

Epoch 00165: val_loss did not improve from 0.02607
Epoch 166/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0696 - accuracy: 0.9801 - val_loss: 0.0506 - val_accuracy: 0.9831

Epoch 00166: val_loss did not improve from 0.02607
Epoch 167/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0743 - accuracy: 0.9778 - val_loss: 0.0538 - val_accuracy: 0.9808

Epoch 00167: val_loss did not improve from 0.02607
Epoch 168/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0673 - accuracy: 0.9772 - val_loss: 0.0423 - val_accuracy: 0.9854

Epoch 00168: val_loss did not improve from 0.02607
Epoch 169/100000
11/11 [==

11/11 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.9808 - val_loss: 0.0498 - val_accuracy: 0.9823

Epoch 00205: val_loss did not improve from 0.02607
Epoch 206/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0653 - accuracy: 0.9798 - val_loss: 0.0621 - val_accuracy: 0.9777

Epoch 00206: val_loss did not improve from 0.02607
Epoch 207/100000
11/11 [==============================] - 0s 12ms/step - loss: 0.0652 - accuracy: 0.9810 - val_loss: 0.0362 - val_accuracy: 0.9885

Epoch 00207: val_loss did not improve from 0.02607
Epoch 208/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0681 - accuracy: 0.9777 - val_loss: 0.0414 - val_accuracy: 0.9862

Epoch 00208: val_loss did not improve from 0.02607
Epoch 209/100000
11/11 [==============================] - 0s 7ms/step - loss: 0.0679 - accuracy: 0.9802 - val_loss: 0.0875 - val_accuracy: 0.9738

Epoch 00209: val_loss did not improve from 0.02607
Epoch 210/100000
11/11 [==

11/11 [==============================] - 0s 6ms/step - loss: 0.0643 - accuracy: 0.9800 - val_loss: 0.0753 - val_accuracy: 0.9762

Epoch 00246: val_loss did not improve from 0.02289
Epoch 247/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0660 - accuracy: 0.9819 - val_loss: 0.0325 - val_accuracy: 0.9900

Epoch 00247: val_loss did not improve from 0.02289
Epoch 248/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0654 - accuracy: 0.9798 - val_loss: 0.0416 - val_accuracy: 0.9877

Epoch 00248: val_loss did not improve from 0.02289
Epoch 249/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0660 - accuracy: 0.9802 - val_loss: 0.0685 - val_accuracy: 0.9777

Epoch 00249: val_loss did not improve from 0.02289
Epoch 250/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0630 - accuracy: 0.9800 - val_loss: 0.0381 - val_accuracy: 0.9892

Epoch 00250: val_loss did not improve from 0.02289
Epoch 251/100000
11/11 [===

11/11 [==============================] - 0s 6ms/step - loss: 0.0634 - accuracy: 0.9828 - val_loss: 0.0242 - val_accuracy: 0.9923

Epoch 00287: val_loss did not improve from 0.02005
Epoch 288/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0682 - accuracy: 0.9793 - val_loss: 0.0383 - val_accuracy: 0.9885

Epoch 00288: val_loss did not improve from 0.02005
Epoch 289/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.9853 - val_loss: 0.0692 - val_accuracy: 0.9800

Epoch 00289: val_loss did not improve from 0.02005
Epoch 290/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.9805 - val_loss: 0.0232 - val_accuracy: 0.9931

Epoch 00290: val_loss did not improve from 0.02005
Epoch 291/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0583 - accuracy: 0.9792 - val_loss: 0.0398 - val_accuracy: 0.9869

Epoch 00291: val_loss did not improve from 0.02005
Epoch 292/100000
11/11 [===

11/11 [==============================] - 0s 6ms/step - loss: 0.0568 - accuracy: 0.9824 - val_loss: 0.0343 - val_accuracy: 0.9908

Epoch 00328: val_loss did not improve from 0.02005
Epoch 329/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0536 - accuracy: 0.9828 - val_loss: 0.0547 - val_accuracy: 0.9831

Epoch 00329: val_loss did not improve from 0.02005
Epoch 330/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0467 - accuracy: 0.9859 - val_loss: 0.0774 - val_accuracy: 0.9762

Epoch 00330: val_loss did not improve from 0.02005
Epoch 331/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0609 - accuracy: 0.9827 - val_loss: 0.0288 - val_accuracy: 0.9923

Epoch 00331: val_loss did not improve from 0.02005
Epoch 332/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0488 - accuracy: 0.9842 - val_loss: 0.0390 - val_accuracy: 0.9869

Epoch 00332: val_loss did not improve from 0.02005
Epoch 333/100000
11/11 [===

11/11 [==============================] - 0s 6ms/step - loss: 0.0597 - accuracy: 0.9799 - val_loss: 0.0256 - val_accuracy: 0.9923

Epoch 00369: val_loss did not improve from 0.02005
Epoch 370/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0565 - accuracy: 0.9822 - val_loss: 0.0348 - val_accuracy: 0.9900

Epoch 00370: val_loss did not improve from 0.02005
Epoch 371/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0531 - accuracy: 0.9845 - val_loss: 0.0506 - val_accuracy: 0.9838

Epoch 00371: val_loss did not improve from 0.02005
Epoch 372/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0564 - accuracy: 0.9841 - val_loss: 0.0408 - val_accuracy: 0.9862

Epoch 00372: val_loss did not improve from 0.02005
Epoch 373/100000
11/11 [==============================] - 0s 6ms/step - loss: 0.0511 - accuracy: 0.9855 - val_loss: 0.0340 - val_accuracy: 0.9915

Epoch 00373: val_loss did not improve from 0.02005
Epoch 374/100000
11/11 [===

In [11]:
# 학습 완료된 모델을 로딩한다
best_model = load_model('models/275-0.020053867250680923.hdf5')
best_model

In [12]:
# 예측결과 추출한다.
pred = (best_model.predict(x) > 0.5).astype('int32')
pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [13]:
r1 = accuracy_score(y, pred)
r1

0.9782976758503925